In [14]:
import pandas
import sqlalchemy

import server.model.connection as sm_connection
import server.model.event as sm_event
import server.model.match as sm_match
import server.view.dataframes as sv_dataframes
import server.view.excel as sv_excel

In [15]:
sm_event.EventDal.set_current_event("test_holoviews", "1318")
sm_event.EventDal.set_current_match("112-q")

'current match 112-q'

In [29]:
def measure_summary_df(num_matches=12):
    # Connect to database
    conn = sm_connection.engine.connect()

    # Get current event
    evt_id = sm_event.EventDal.get_current_event()[0]

    select_sum = sqlalchemy.text(
        "WITH current AS (SELECT status.event_id as event_id, status.match, schedules.date "
        "FROM status INNER JOIN schedules ON status.event_id=schedules.event_id AND status.match=schedules.match "
        "WHERE date <> 'na' LIMIT 1), "

        "recent_matches as ( SELECT * FROM ( "
        "SELECT row_number() over (partition by team order by sched.date desc) as r, "
        " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
        " row_schedule WHERE row_schedule.r <= " + str(
            num_matches) + " ORDER by team, date desc), "

       "team_match_count as ( "
       "SELECT team, count(team) AS team_matches FROM recent_matches GROUP BY team"
       ") "
        
       "SELECT teams.name AS team, phases.name AS phase, tasks.name AS task, actors.name AS actor, "
       "MAX(team_match_count.team_matches) AS matches, "
       "SUM(successes) AS sum_successes, MAX(successes) as max_successes, MIN(successes) as min_successes, "
       "COUNT(successes) AS count_successes, AVG(successes) as avg_successes, "
       "SUM(attempts) AS sum_attempts, MAX(attempts) as max_attempts, MIN(attempts) AS min_attempts, "
       "COUNT(attempts) AS count_attempts, AVG(attempts) as avg_attempts, "
       "MAX(cycle_times) AS max_cycle_times, MIN(cycle_times) AS min_cycle_times, "
       "AVG(cycle_times) AS avg_cycle_times, COUNT(cycle_times) AS count_cycle_times "
       "FROM (((((((teams FULL OUTER JOIN measures ON teams.id=measures.team_id) "
       "LEFT JOIN tasks ON tasks.id = measures.task_id) "
       "LEFT JOIN phases ON phases.id = measures.phase_id) "
       "LEFT JOIN events ON events.id = measures.event_id) "
       "LEFT JOIN actors ON actors.id = measures.actor_id) "
       "LEFT JOIN matches ON matches.id = measures.match_id) "
       "LEFT JOIN team_match_count ON team_match_count.team = teams.name) "
       "RIGHT JOIN recent_matches ON recent_matches.match = matches.name AND team_match_count.team = teams.name "
       "AND recent_matches.team = team_match_count.team "
       "WHERE events.id = :evt_id "
       "GROUP BY teams.name, tasks.name, phases.name, actors.name "
       "ORDER BY teams.name, phases.name, tasks.name, actors.name;"
    ).bindparams(evt_id=evt_id)
    summ_df = pandas.read_sql(select_sum, conn)
    summ_df.set_index(['team', 'phase', 'actor', 'task'], inplace=True)
    conn.close()
    return summ_df

In [32]:
df = measure_summary_df(12)
df.head(24)

matches  sum_successes  max_successes  \
team phase  actor task                                                   
108  auto   robot placeScale          10             21              5   
                  placeSwitch         10              1              1   
                  startPosition       10              0              0   
     teleop robot placeScale          10             17              3   
                  placeSwitch         10             56             12   
1165 auto   robot placeScale          10             18              5   
                  placeSwitch         10              2              1   
                  startPosition       10              0              0   
     teleop robot placeScale          10             16              4   
                  placeSwitch         10             57             13   
1287 auto   robot placeScale          10             12              2   
                  placeSwitch         10              3              1   
                  startPosition       10              0              0   
     teleop robot placeScale          10             16              5   
                  placeSwitch         10             39              7   
1296 auto   robot placeScale          10             13              3   
                  placeSwitch         10              4              1   
                  startPosition       10              0              0   
     teleop robot placeScale          10             24              6   
                  placeSwitch         10             69             10   
1318 auto   robot placeScale          10             18              4   
                  placeSwitch         10              3              1   
                  startPosition       10              0              0   
     teleop robot placeScale          10             14              3   

                                 min_successes  count_successes  \
team phase  actor task                                            
108  auto   robot placeScale                 1               10   
                  placeSwitch                0                4   
                  startPosition              0               10   
     teleop robot placeScale                 0               10   
                  placeSwitch                1               10   
1165 auto   robot placeScale                 0               10   
                  placeSwitch                1                2   
                  startPosition              0               10   
     teleop robot placeScale                 0               10   
                  placeSwitch                1               10   
1287 auto   robot placeScale                 0               10   
                  placeSwitch                0                5   
                  startPosition              0               10   
     teleop robot placeScale                 1               10   
                  placeSwitch                2               10   
1296 auto   robot placeScale                 0               10   
                  placeSwitch                0                6   
                  startPosition              0               10   
     teleop robot placeScale                 0               10   
                  placeSwitch                2               10   
1318 auto   robot placeScale                 1               10   
                  placeSwitch                0                5   
                  startPosition              0               10   
     teleop robot placeScale                 0               10   

                                 avg_successes  sum_attempts  max_attempts  \
team phase  actor task                                                       
108  auto   robot placeScale          2.100000            44             7   
                  placeSwitch         0.250000             4             1   
                  startPosition       0.000000       

In [36]:
dfr = sv_dataframes.ranking_df(12)
dfr["auto"]["robot"]["placeSwitch"]

,matches,sum_successes,max_successes,min_successes,count_successes,avg_successes,sum_attempts,max_attempts,min_attempts,count_attempts,avg_attempts,max_cycle_times,min_cycle_times,avg,count_cycle_times
team,,,,,,,,,,,,,,,
1540,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0
1884,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1983,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
2046,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0
4468,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
4738,1.0,4.0,4.0,4.0,1.0,4.0,4.0,4.0,4.0,1.0,4.0,0.0,0.0,0.0,1.0
